# 4. Setting up the IoT Edge device

## 4.1. Prerequisites
- An edge device
    - In this sample we will create a Virtual Machine (VM) in your Azure cloud subscription
    - If you want to use your own edge device (a physical PC), follow the steps below (omitting the create VM section) and install below same software (docker engines, iotedge runtime etc.) and approapriate Nvidia GPU driver on your device.
- An NVidia GPU accelerator (On Azure, N series Virtual Machines have GPU card, so we will create an N series VM)

## 4.2. Get global variables
We will read the previously stored variables## 3.1. Get global variables
We will read the previously stored variables

In [ ]:
from dotenv import set_key, get_key, find_dotenv
envPath = find_dotenv(raise_error_if_not_found=True)

resourceLocation = get_key(envPath, "resourceLocation")
resourceGroupName = get_key(envPath, "resourceGroupName")
iotEdgeDeviceConnString = get_key(envPath, "iotEdgeDeviceConnString")
# We will use IoT Edge Device ID also as VM name
iotDeviceId = get_key(envPath, "iotDeviceId")

# We get current username so will use it to create SSH keys for connecting the VM
import os
userName = os.environ['USER']

## 4.3. Create a Virtual Machine with GPU
With following steps, we will execute a shell script to create an Ubuntu VM in your Azure subscription with required libraries installed.  

### 4.3.1 Set parameters to be used for creating the Virtual Machine
https://docs.microsoft.com/en-us/azure/virtual-machines/windows/quick-create-cli

In [ ]:
# Select VM size per your speed requirements. https://docs.microsoft.com/en-us/azure/virtual-machines/sizes-general 
# NC-series VMs are powered by the NVIDIA Tesla K80 card and the Intel Xeon E5-2690 v3 (Haswell) processor. 
vm_size="Standard_NC6"

dns_name = iotDeviceId
public_ip_name = iotDeviceId + 'publicip'
vnet_name=iotDeviceId + 'vnet'
subnet_name=iotDeviceId + 'subnet'
vnet_prefix="192.168.0.0/16"
subnet_name="FrontEnd"
subnet_prefix="192.168.1.0/24"
nsg_name=iotDeviceId + 'nsg'
nic_name=iotDeviceId + 'nic'

# Static DNS name of the VM.
vm_dns_name= iotDeviceId + "." + resourceLocation + ".cloudapp.azure.com"

In [ ]:
%%bash --out output -s "$iotDeviceId" "$userName" "$vm_dns_name"

# -----------------------
# Create SSH Keys - !!! DONT RUN THIS CELL AFTER THE VM CREATED. IT WILL RESET THE PUBLIC KEYS!!!
# -----------------------
# Create public/private ssh keys.
yes y | ssh-keygen -b 2048 -t rsa -f ~/.ssh/$1"_id_rsa" -C $2@$3 -q -N '' >/dev/null

# Remove prev. trusted host with same name (IP may be changed)
if [ -e ~/.ssh/known_hosts ]; then
ssh-keygen -f ~/.ssh/known_hosts -R $3
fi

In [ ]:
%%bash -s "$resourceGroupName" "$resourceLocation" "$public_ip_name" "$dns_name" "$vnet_name" "$vnet_prefix" "$subnet_name" "$subnet_prefix" "$nsg_name" "$nic_name" "$iotDeviceId" "$vm_size"

# Create resource group
az group create                                                                             \
    --name "$1"                                                                             \
    --location "$2"

# Create a public IP address resource with a static IP address
az network public-ip create                                                                 \
   --name "$3"                                                                              \
   --resource-group "$1"                                                                    \
   --location "$2"                                                                          \
   --allocation-method Static                                                               \
   --dns-name "$4"                                                                                   

# Create a virtual network with one subnet
az network vnet create                                                                      \
   --name "$5"                                                                              \
   --resource-group "$1"                                                                    \
   --location "$2"                                                                          \
   --address-prefix "$6"                                                                    \
   --subnet-name "$7"                                                                       \
   --subnet-prefix "$8"                                                                         


az network nsg create                                                                       \
   --name "$9"                                                                              \
   --resource-group "$1"                                                                  

# Open SSH port
az network nsg rule create                                                                  \
   --resource-group "$1"                                                                    \
   --nsg-name "$9"                                                                          \
   --name "Default SSH"                                                                     \
   --destination-port-ranges 22                                                             \
   --protocol Tcp                                                                           \
   --access Allow                                                                           \
   --priority 1020

# Create a network interface connected to the VNet with a static private IP address 
# and associate the public IP address resource to the NIC.
az network nic create                                                                       \
   --name "${10}"                                                                           \
   --resource-group "$1"                                                                    \
   --location "$2"                                                                          \
   --subnet "$7"                                                                            \
   --vnet-name "$5"                                                                         \
   --public-ip-address "$3"                                                                 \
   --network-security-group "$9"                                                                     

# Create the VM
# For image param: Ubuntu 16.04-LTS image -> https://docs.microsoft.com/en-us/azure/virtual-machines/linux/cli-ps-findimage
# Instead of Password login, we are uploading Private SSH key. Update the path accrodingly if needed
az vm create                                                                                \
   --name "${11}"                                                                           \
   --resource-group "$1"                                                                    \
   --location "$2"                                                                          \
   --storage-sku Standard_LRS                                                               \
   --os-disk-name "${11}""_osdisk"                                                          \
   --image "Canonical:UbuntuServer:18.04-LTS:latest"                                        \
   --size "Standard_NC6"                                                                    \
   --nics "${10}"                                                                           \
   --authentication-type ssh                                                                \
   --ssh-key-value "$(< ~/.ssh/"${11}"_id_rsa.pub)"

The above command may take a couple minutes for the creation of VMs and services. Once it's done, you can go to Azure Portal and check the resources. Below is an Azure Portal snapshot with all services created in the command above. 
<img src="documents/_iotedgedevice_portal.png" alt="iotedgedeviceportal" width="1280">  

In [ ]:
%%bash -s "$iotDeviceId" "$userName" "$vm_dns_name"

# Install ssh keys to remote machine
ssh -i ~/.ssh/$1"_id_rsa" -o "StrictHostKeyChecking no" $2@$3 "echo 'SSH key transferred.'"

### 4.3.3 Install NVidia Cuda Drivers for Tesla K80
https://docs.microsoft.com/en-us/azure/virtual-machines/linux/n-series-driver-setup

Execution of below cell may take few minutes.

In [ ]:
%%bash -s "$iotDeviceId" "$userName" "$vm_dns_name"

ssh -i ~/.ssh/$1"_id_rsa" $2@$3 << EOF

# Install driver
wget -O /tmp/cuda-repo-ubuntu1804_10.0.130-1_amd64.deb http://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-repo-ubuntu1804_10.0.130-1_amd64.deb 
sudo dpkg -i /tmp/cuda-repo-ubuntu1804_10.0.130-1_amd64.deb
sudo apt-key adv --fetch-keys https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/7fa2af80.pub 
rm -f /tmp/cuda-repo-ubuntu1804_10.0.130-1_amd64.deb
sudo apt-get update
sudo apt-get install cuda-drivers

# Restart the VM
sudo -b bash -c 'sleep 5; reboot' &>/dev/null;

EOF

In this step we will wait few minutes because in the previous cell we restarted the VM and will wait it to reboot.

### 4.3.4 Install Docker Engine on Ubuntu
https://docs.docker.com/engine/install/ubuntu/


In [ ]:
%%bash -s "$iotDeviceId" "$userName" "$vm_dns_name"

ssh -i ~/.ssh/$1"_id_rsa" $2@$3 << EOF

sudo apt-get update

sudo apt-get install -y \
    apt-transport-https \
    ca-certificates \
    curl \
    gnupg-agent \
    software-properties-common

curl -fsSL https://download.docker.com/linux/ubuntu/gpg | sudo apt-key add -

sudo apt-key fingerprint 0EBFCD88

sudo add-apt-repository "deb [arch=amd64] https://download.docker.com/linux/ubuntu bionic stable"

sudo apt-get update

sudo apt-get install -y docker-ce docker-ce-cli containerd.io

EOF

### 4.3.5 Install NVIDIA Container Toolkit

https://github.com/NVIDIA/nvidia-docker#upgrading-with-nvidia-docker2-deprecated

!!! Important !!! Take care about following note on "nvidia-docker" website:

> Note that in the future, nvidia-docker2 packages will no longer be supported.

Since current version of IoT Edge Runtime (1.0.9) supports this V2, we are going with it for now.

In [ ]:
%%bash  -s "$iotDeviceId" "$userName" "$vm_dns_name"

ssh -i ~/.ssh/$1"_id_rsa" $2@$3 << EOF

curl -s -L https://nvidia.github.io/nvidia-docker/gpgkey | sudo apt-key add -
curl -s -L https://nvidia.github.io/nvidia-docker/ubuntu18.04/nvidia-docker.list | sudo tee /etc/apt/sources.list.d/nvidia-docker.list

sudo apt-get update

sudo apt-get install -y nvidia-docker2

sudo systemctl restart docker

EOF

### Test the installation and access to GPU within a container

In [ ]:
%%bash  -s "$iotDeviceId" "$userName" "$vm_dns_name"
ssh -i ~/.ssh/$1"_id_rsa" $2@$3 << EOF

# sudo docker run --gpus all nvidia/cuda:10.0-base nvidia-smi
sudo docker run --runtime nvidia nvidia/cuda:10.0-base nvidia-smi

EOF

### 4.3.6 Install the Azure IoT Edge runtime


In [ ]:
%%bash  -s "$iotDeviceId" "$userName" "$vm_dns_name"
ssh -i ~/.ssh/$1"_id_rsa" $2@$3 << EOF

sudo apt-get -y install curl

curl https://packages.microsoft.com/config/ubuntu/18.04/multiarch/prod.list > ./microsoft-prod.list
sudo cp ./microsoft-prod.list /etc/apt/sources.list.d/
curl https://packages.microsoft.com/keys/microsoft.asc | gpg --dearmor > microsoft.gpg
sudo cp ./microsoft.gpg /etc/apt/trusted.gpg.d/
sudo apt-get -y update
sudo apt-get -y install iotedge

EOF

### 4.3.7. Set up the configuration file of IoT Edge Runtime

With below command, IoT Edge Runtime in the VM will be configured.

In [ ]:
%%bash  -s "$iotDeviceId" "$userName" "$vm_dns_name" "$iotEdgeDeviceConnString"
ssh -i ~/.ssh/$1"_id_rsa" $2@$3 << EOF

sudo sed -i "s#\(device_connection_string: \).*#\1\'$4\'#g" /etc/iotedge/config.yaml

sudo systemctl restart iotedge

EOF

### Restart the VM

In [ ]:
%%bash  -s "$iotDeviceId" "$userName" "$vm_dns_name"
ssh -i ~/.ssh/$1"_id_rsa" $2@$3 << EOF
sudo -b bash -c 'sleep 5; reboot' &>/dev/null;
EOF

### Get SSH connection string

In [ ]:
# If needed, use the output command to SSH into the VM
!echo ssh -i ~/.ssh/$iotDeviceId"_id_rsa" $userName@$vm_dns_name